# Análisis de algoritmos

## Apartado a

Mejor caso y $\Omega$: Si `par(C[i,1])` es `true` para `i=1`, y luego `par(C[i,j])` es true (para `i=1` y `j=2`), `encontrado` es verdadero y el algoritmo termina tras unas pocas instrucciones, porque ambos bucles terminan cuando `encontrado==true`. Eso sería el mejor caso, que daría que $t(n)\in \Omega(1)$.


Peor caso y $O$: Se da cuando nunca se ejecuta `encontrado` = verdadero, lo que ocurre por ejemplo cuando el array `C` está formado por números impares. En ese caso $t(n)$ pertenece a $O(n\log(n))$, el log corresponde al bucle externo (se actualiza con `i=i*2`) y la `n` al bucle interno (`j=j+1`, `j=2…n`).

## Apartado b

Suponiendo $n$ potencia de $2$: $n=2^k$, $k=\log_2(n)$

$t’(k) – 5t’(k-1) – 6t’(k-2) = 2^k$

$(x^2 – 5x – 6)(x-2) = 0$ con raíces $x=-1,6,2$

$t(n) = c_1(-1)^{\log_2(n)} + c_2n^{2,6} + c_3n$

$n=2,4,8$

# DyV

La idea princial es darse cuenta de que en el centro existen dos cadenas que cruzan.

In [5]:
# función principal
def mayor_bache(S, l, r):
    if r - l + 1 < 3:
        return None  # No hay tripleta posible

    m = (l + r) // 2

    # Mejor tripleta en la izquierda
    izquierda = mayor_bache(S, l, m)
    # Mejor tripleta en la derecha
    derecha = mayor_bache(S, m + 1, r)
    # Mejor tripleta cruzando
    cruzada = mejor_tripleta_cruzada(S, l, m, r)

    # Elegir la mejor de las tres
    candidatos = [t for t in [izquierda, derecha, cruzada] if t is not None]
    if not candidatos:
        return None
    return max(candidatos, key=lambda t: S[t[0]] + S[t[2]] - S[t[1]])

def mejor_tripleta_cruzada(S, l, m, r):
    i = m - 1
    t1 = None
    if i >= l and i + 2 <= r:
        if S[i+1] < S[i] and S[i+1] < S[i+2]:
            t1 = (i, i+1, i+2)

    i = m
    t2 = None
    if i >= l and i + 2 <= r:
        if S[i+1] < S[i] and S[i+1] < S[i+2]:
            t2 = (i, i+1, i+2)

    # Escoger la mejor
    tuplas = [t for t in [t1, t2] if t is not None]
    if not tuplas:
        return None
    else:
        return max(tuplas, key=lambda t: S[t[0]] + S[t[2]] - S[t[1]])

def resolver_mayor_bache(S):
    resultado = mayor_bache(S, 0, len(S) - 1)
    if resultado:
        i, j, k = resultado
        valor = S[i] + S[k] - S[j]
        return (i, j, k, valor)
    else:
        return None

# Ejemplo de uso:
S=[1,3,2,2,1,4,1,4,3,5,8,5,5,4]
print(resolver_mayor_bache(S))

(5, 6, 7, 7)


# Voraces

La decisión voraz: escoger la casilla de menor peso adyacente. No devuelve la solución óptima, bastaba usar el ejemplo del examen.

In [8]:
def greedy_path(grid):
    n = len(grid)
    i, j = 0, 0
    path = [(i, j)]
    total_cost = grid[i][j]
    
    while i < n - 1 or j < n - 1:
        moves = []
        # Mover hacia abajo
        if i < n - 1:
            moves.append((grid[i + 1][j], (i + 1, j)))
        # Mover a la derecha
        if j < n - 1:
            moves.append((grid[i][j + 1], (i, j + 1)))
        # Mover en diagonal (abajo y derecha)
        if i < n - 1 and j < n - 1:
            moves.append((grid[i + 1][j + 1], (i + 1, j + 1)))
        
        # Elegir el movimiento con el costo mínimo
        cost, (next_i, next_j) = min(moves, key=lambda x: x[0])
        path.append((next_i, next_j))
        total_cost += cost
        i, j = next_i, next_j
        
    return path, total_cost


In [11]:
grid = [
    [3, 3, 0, 0],
    [1, 7, 6, 0],
    [1, 2, 24, 1],
    [3, 3, 1, 3]
]
    
path, cost = greedy_path(grid)
print("Ruta encontrada:", path)
print("Costo total:", cost)

Ruta encontrada: [(0, 0), (1, 0), (2, 0), (2, 1), (3, 2), (3, 3)]
Costo total: 11


# PD

La ecuación de recurrencia es:
$$
dp[i][j] = \texttt{grid[i][j]} + \min
\begin{cases}
dp[i-1][j] & \text{si } i > 0 \\
dp[i][j-1] & \text{si } j > 0 \\
dp[i-1][j-1] & \text{si } i > 0 \text{ y } j > 0
\end{cases}
$$

Con el caso base:

$$
dp[0][0] = \texttt{grid[0][0]}
$$

In [19]:
def min_cost_dp(grid):
    n = len(grid)
    dp = [[float('inf')] * n for _ in range(n)]
    dp[0][0] = grid[0][0]
    
    for i in range(n):
        for j in range(n):
            if i > 0:
                dp[i][j] = min(dp[i][j], dp[i-1][j] + grid[i][j])
            if j > 0:
                dp[i][j] = min(dp[i][j], dp[i][j-1] + grid[i][j])
            if i > 0 and j > 0:
                dp[i][j] = min(dp[i][j], dp[i-1][j-1] + grid[i][j])

    return dp, dp[n-1][n-1]
    

In [20]:
grid = [
    [3, 3, 0, 0],
    [1, 7, 6, 0],
    [1, 2, 24, 1],
    [3, 3, 1, 3]
]
    
dp, cost = min_cost_dp(grid)
print("Costo total:", cost)

Costo total: 10


In [21]:
def reconstruir_ruta(dp):
    n = len(dp)
    i, j = n - 1, n - 1
    path = [(i, j)]

    while (i, j) != (0, 0):
        opciones = []
        if i > 0:
            opciones.append(((i - 1, j), dp[i - 1][j]))
        if j > 0:
            opciones.append(((i, j - 1), dp[i][j - 1]))
        if i > 0 and j > 0:
            opciones.append(((i - 1, j - 1), dp[i - 1][j - 1]))

        # Elegimos la celda anterior con menor coste acumulado
        (i, j), _ = min(opciones, key=lambda x: x[1])
        path.append((i, j))

    path.reverse()
    return path

In [22]:
reconstruir_ruta(dp)

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (3, 3)]

# Backtracking

La idea más limpia que hace uso del esquema visto en clase consiste en llevar la posición actual y que valor se encargue de calcular el peso del camino.

In [12]:
def solucion(i, j, n):
    return i == n-1 and j == n-1

def criterio(i, j, n):
    if i >= n or j >= n:
        return False
    return i != n-1 or j != n-1

def valor(s, grid):
    i_actual = 0
    j_actual = 0
    b_actual = grid[0][0]
    for d in s:
        if d == -1:
            break
        if d == 0:
            i_actual += 1
        elif d == 1:
            j_actual += 1
        else:
            i_actual += 1
            j_actual += 1
        b_actual += grid[i_actual][j_actual]
    return b_actual

def backtracking(grid):
    s = [-1]*(2*len(grid)) #0 abajo, 1 derecha, 2 diagonal
    i_actual = 0
    j_actual = 0
    n = len(grid)

    voa = float("inf")
    soa = None
    nivel = 1

    while nivel != 0:
        # genero un hermano
        s[nivel-1] += 1
        if s[nivel-1] == 0:
            i_actual += 1
        elif s[nivel-1] == 1:
            i_actual -= 1
            j_actual += 1
        else:
            j_actual -= 1
            i_actual += 1
            j_actual += 1


        if solucion(i_actual, j_actual, n) and valor(s, grid) < voa:
            voa = valor(s, grid)
            soa = s.copy()

        if criterio(i_actual, j_actual, n):
            nivel += 1
        else:
            while nivel > 0 and (not s[nivel-1] < 2):
                if s[nivel-1] == 0:
                    i_actual -= 1
                elif s[nivel-1] == 1:
                    j_actual -= 1
                else:
                    i_actual -= 1
                    j_actual -= 1
                s[nivel-1] = -1
                nivel -= 1
    return soa, voa

In [13]:
grid = [
    [3, 3, 0, 0],
    [1, 7, 6, 0],
    [1, 2, 24, 1],
    [3, 3, 1, 3]
]

In [14]:
backtracking(grid)

([0, 2, 2, 1, -1, -1, -1, -1], 10)